In [19]:
# Model 클래스 상속

In [20]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [21]:
# 1) 컬럼명 정의 (마지막이 레이블 target)
cols = [
    "area", "perimeter", "compactness",
    "length", "width", "asymmetry", "groove_length",
    "target"
]

# 2) 데이터 로드
df = pd.read_csv('./seeds_dataset.txt', sep=r'\s+', names=cols)


In [22]:
# 3) 특성과 레이블 분리
X = df.drop("target", axis=1).values      # (210, 7)
y = df["target"].values - 1               # 1,2,3 → 0,1,2

In [23]:
# 4) 학습/테스트 셋 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [24]:
# 5) 특성 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [25]:
# 6) 텐서로 변환
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
X_test  = tf.convert_to_tensor(X_test,  dtype=tf.float32)
y_test  = tf.convert_to_tensor(y_test,  dtype=tf.int32)


In [26]:
# 7) Model 서브클래싱 정의
class SimpleMLP(Model):
    def __init__(self, num_classes):
        super(SimpleMLP, self).__init__(name='mlp')
        # 은닉층: 32 유닛, sigmoid
        self.dense1 = Dense(32, activation='sigmoid', name="hidden1")
        # 출력층: num_classes 유닛, softmax (다중 분류)
        self.dense2 = Dense(num_classes, activation='softmax', name="output")

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

In [27]:
# 8) 모델 생성
model_sub = SimpleMLP(num_classes=3)


In [28]:
# 9) 컴파일
model_sub.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [29]:
# 10) 학습
history = model_sub.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=16,
    verbose=1
)

Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.3618 - loss: 1.0492 - val_accuracy: 0.4412 - val_loss: 1.0493
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4960 - loss: 1.0115 - val_accuracy: 0.5882 - val_loss: 0.9821
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7714 - loss: 0.9307 - val_accuracy: 0.6765 - val_loss: 0.9250
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8547 - loss: 0.8784 - val_accuracy: 0.7353 - val_loss: 0.8759
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.9006 - loss: 0.8316 - val_accuracy: 0.8235 - val_loss: 0.8319
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.8657 - loss: 0.8041 - val_accuracy: 0.9118 - val_loss: 0.7848
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9045 - loss: 0.7741 - val_accuracy: 0.9118 - val_loss: 0.7449
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.8393 - loss: 0.7549 - val_accuracy: 0.9118 - val_loss: 0.7078


In [30]:
# 11) 평가
loss, acc = model_sub.evaluate(X_test, y_test, verbose=0)
print(f"테스트 데이터 정확도: {acc:.4f}")


테스트 데이터 정확도: 0.8571
